# imports 

In [2]:
#imports
import lingpy
from levenshtein_dist_calc import LevenshteinDistanceCalculator
import nltk
from read_tab_files import TabFileReader
import bcubed
from collections import defaultdict

# Clustering

## testing

In [3]:
barb_cognacy = TabFileReader.tab_reader(
    "chl2024_barbacoandata/chl2023_barbacoan_cognacy.tab"
)
barb_forms = TabFileReader.tab_reader(
    "chl2024_barbacoandata/chl2023_barbacoan_forms.tab"
)

In [42]:
barb_word_list = TabFileReader.get_word_list(barb_forms)
barb_alphabet = TabFileReader.get_alphabet(barb_word_list)

In [4]:
taxa = list(barb_forms.iloc[0][1:])
taxa

['Awa Pit', "Cha'palaa", 'Guambiano', 'Totoro', 'Tsafiki']

In [5]:
cognates = list(barb_forms.iloc[2][1:])
cognates

['pil', 'tu', 'pirɨ', 'pirɨ', 'to']

In [21]:
#calculate distance matrix from cognate list (integrate with levenshtein distance calc maybe)
def get_distancematrix(cognate_list):
    distance_matrix = [[0 for i in range(len(cognate_list))] 
                           for j in range(len(cognate_list))]
    for i, source in enumerate(cognate_list):
        for j, target in enumerate(cognate_list):
            if (source == "" 
                or target == ""):  # if one cognate in pair is missing, null value
                distance_matrix[i][j] = float("nan")
            else:
                distance_matrix[i][j] = nltk.edit_distance(source, target)/max(len(source),len(target))
    
    return distance_matrix

In [7]:
distance_matrix = get_distancematrix(cognates)

[[0.0, 1.0, 0.5, 0.5, 1.0],
 [1.0, 0.0, 1.0, 1.0, 0.5],
 [0.5, 1.0, 0.0, 0.0, 1.0],
 [0.5, 1.0, 0.0, 0.0, 1.0],
 [1.0, 0.5, 1.0, 1.0, 0.0]]

In [12]:
cluster_dict_p2 = lingpy.flat_upgma(.2,distance_matrix,taxa)
cluster_dict_p5 = lingpy.flat_upgma(.5,distance_matrix,taxa)
print(cluster_dict_p2)
print(cluster_dict_p5)

{0: ['Awa Pit'], 1: ["Cha'palaa"], 2: ['Guambiano', 'Totoro'], 4: ['Tsafiki']}
{0: ['Awa Pit', 'Guambiano', 'Totoro'], 1: ["Cha'palaa", 'Tsafiki']}


In [11]:
gold_clusters = list(barb_cognacy.iloc[2][1:])
gold_clusters

['53', '45', '53', '53', '45']

In [19]:
#gold file clustering to taxa:cluster dictionary transformer for bcubed comparisons
def gold_clust2taxa_dict(taxa, clusters):
    cluster_dict = {}
    for lang, cluster in zip(taxa, clusters):
        cluster_dict[lang] = set([cluster])
    return cluster_dict

gold_taxadict = gold_clust2taxa_dict(taxa, gold_clusters)

In [17]:
#lingpy.upgma_flat output to taxa:cluster dictionary transformer for bcubed comparisons
def upgma2taxa_dict(calculated_dict):
    output_clusters = {}
    for clus, langs in calculated_dict.items():
        for lang in langs:
            output_clusters[lang] = set(str(clus))
    return output_clusters

p2_taxa_dict = upgma2taxa_dict(cluster_dict_p2)
p5_taxa_dict = upgma2taxa_dict(cluster_dict_p5)

In [20]:
precision = bcubed.precision(p2_taxa_dict, gold_taxadict)
recall = bcubed.recall(p2_taxa_dict, gold_taxadict)
fscore = bcubed.fscore(precision, recall)
print(precision, recall)

1.0 0.5333333333333333


In [ ]:
def prepare_comparison(threshold:float, gold_row:list, cognates:list, taxa:list)->tuple:
    distance_matrix = get_distancematrix(cognates)
    cluster_dict = lingpy.flat_upgma(threshold,distance_matrix,taxa)
    taxa_dict = upgma2taxa_dict(cluster_dict)
    gold_taxadict = gold_clust2taxa_dict(taxa, gold_row)
    return taxa_dict, gold_taxadict


In [ ]:
def 